In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pathlib

## Mapping esg_questions to principle and esg_category

In [8]:
# Strating from esg_class file  that has ElementName, Principles, esg_category
ques = pd.read_csv(r"C:\Users\Dell\Desktop\esg_class.csv")
ques = ques.dropna()
dc_ques_category = {}
for element, category, principle in zip(ques["Element Name"].str.lower(), ques["esg_category"], ques["Principles"]):
    dc_ques_category[element] = [principle, category]

In [12]:
# Adding principle and esg_category in each file columns 
# Accessing each xbrl file and adding Principle and esg_category column and labeling them according to the standard unique questions
path = r"C:\Users\Dell\Desktop\temp_esg"   # Path to the xbrl file folder

directory = pathlib.Path(path)
txt_files = directory.glob("*.xlsx") 
i=0
for file in tqdm(txt_files):
    df = pd.read_excel(file)
    df["Principles"] = df["Element Name"].str.lower().apply(lambda x: dc_ques_category[x][0] if x in dc_ques_category.keys() else np.nan)
    df["esg_category"] = df["Element Name"].str.lower().apply(lambda x: dc_ques_category[x][1] if x in dc_ques_category.keys() else np.nan)
    df.to_excel(file, index=False)  # Saving the modified xbrl's

169it [02:24,  1.17it/s]


## (E, S, G) SCORING bunch of files

In [15]:
path = r"C:\Users\Dell\Desktop\temp_esg"
directory = pathlib.Path(path)
file_path = directory.glob("*.xlsx")

# Function to calculate ESG SCORE
def esg_score(category):
    """
    Calculate the ESG (Environmental, Social, and Governance) score for a given category.

    Args:
        category (str): The category for which the ESG score is calculated.
        Select Category from : ["Environment", "Social", "Governance"]

    Returns:
        float: The calculated ESG score for the specified category, ranging between 0 and 1.
    """
    total = len(df[df.esg_category==category].Score)       # total questions of a category
    sum_of_1 = df[df.esg_category==category].Score.sum()   # count of 1's in score column from the total questions of particular category
    score = round(sum_of_1/total, 3)                       # score will be between 0 and 1.
    return score

# The final dataframe that has company identification no.(CIN), company name, E,S,G SCORE
final_df = pd.DataFrame(columns=["CIN", "COMPANY_NAME", "E_SCORE", "S_SCORE", "G_SCORE"])   

for file in tqdm(file_path):
    df = pd.read_excel(file)
    
    # Calculating E,S,G SCORE for each class
    E_score = esg_score("Environment")
    S_score = esg_score("Social")
    G_score = esg_score("Governance")
    
    try:
        cin = df[df["Element Name"].str.lower()=="corporateidentitynumber"]["Fact Value"][0] # Extracting CIN
    except Exception:
        cin = np.nan
        print(file)
    try:
        company_name = df[df["Element Name"].str.lower()=="nameofthecompany"]["Fact Value"][1] # Extracting Company Name
    except:
        company_name = np.nan
    
    # Appending data for one company at a time in the final df
    data_dc = {"CIN":cin, "COMPANY_NAME":company_name, "E_SCORE":E_score, "S_SCORE":S_score, "G_SCORE":G_score}
    final_df.loc[len(final_df)] = data_dc
final_df
final_df.to_excel(r"C:\Users\Dell\Desktop\esg_score_copy.xlsx", index=False)

122it [00:47,  2.34it/s]C:\Users\Dell\AppData\Local\Temp\ipykernel_10728\2958372495.py:19: RuntimeWarning: invalid value encountered in longlong_scalars
  score = round(sum_of_1/total, 3)                       # score will be between 0 and 1.
123it [00:47,  3.03it/s]

C:\Users\Dell\Desktop\temp_esg\br_rallis_2021_2022_20220800055.xlsx


169it [01:07,  2.52it/s]


,CIN,COMPANY_NAME,E_SCORE,S_SCORE,G_SCORE
0,L31300KA1987PLC013543,3M INDIA LIMITED,0.239,0.135,0.425
1,L65922RJ2011PLC034297,Aavas Financiers Limited,0.180,0.314,0.247
2,L32202KA1949PLC032923,ABB INDIA LIMITED,0.660,0.482,0.521
3,L51100GJ1993PLC019067,Adani Enterprises Limited (“AEL or the Company”),0.643,0.488,0.557
4,L40106GJ2015PLC082007,Adani Green Energy Limited,0.321,0.260,0.490
...,...,...,...,...,...
164,L29150TG1930PLC000576,VST INDUSTRIES LIMITED,0.483,0.378,0.404
165,L27100GJ1995PLC025609,WELSPUN CORP LIMITED,0.498,0.408,0.428
166,L17110GJ1985PLC033271,Welspun India Limited,0.511,0.383,0.381
167,L32102KA1945PLC020800,Wipro Limited,0.676,0.341,0.385


# ABOVE TWO SCRIPTS COMBINED

In [4]:
# This script will extract questions, priciple, category from esg_class.csv
# Then It base_scoring() function takes input as path of esg_raw 169 files and returns a df 
# that has CIN, COMPANY_NAME, 

question = pd.read_csv(r"C:\Users\Dell\Desktop\esg_class.csv")
quest = question.dropna()
dc_ques_category = {}
for element, category, principle in zip(quest["Element Name"].str.lower(), quest["esg_category"], quest["Principles"]):
    dc_ques_category[element] = [principle, category]
    

# Function to add ESG category labels in the file
def categorize(df):
    df["Principles"] = df["Element Name"].str.lower().apply(lambda x: dc_ques_category[x][0] if x in dc_ques_category.keys() else np.nan)
    df["esg_category"] = df["Element Name"].str.lower().apply(lambda x: dc_ques_category[x][1] if x in dc_ques_category.keys() else np.nan)
    return df

# Main function that append company info and calculated ESG score in Final DataFrame
def base_scoring(folder_path):
    
    #Function to calculate ESG score for a given category.
    def esg_score(category):
        """
        input: (str) ESG_category:("Environment", "Social", "Governance")
        output: (float) Score of the input category in a file
        """
        total = len(df[df.esg_category==category].Score)
        sum_of_1 = df[df.esg_category==category].Score.sum() 
        score = round(sum_of_1/total, 3) 
        return score
    
    final_df = pd.DataFrame(columns=["CIN", "COMPANY_NAME", "E_SCORE", "S_SCORE", "G_SCORE"]) 
    
    # Processing each xbrl file at a time
    directory = pathlib.Path(folder_path)
    txt_files = directory.glob("*.xlsx") 
    for file in tqdm(txt_files):
        df = pd.read_excel(file)
        
        try:
            cin = df[df["Element Name"]=="CorporateIdentityNumber"]["Fact Value"][0] # Extracting CIN
        except Exception:
            cin = np.nan
            print(file)
        try:
            company_name = df[df["Element Name"]=="NameOfTheCompany"]["Fact Value"][1] # Extracting Company Name
        except:
            company_name = np.nan
            
        
        df = categorize(df)
        
        E_score = esg_score("Environment")
        S_score = esg_score("Social")
        G_score = esg_score("Governance")
        
        data_dc = {"CIN":cin, "COMPANY_NAME":company_name, "E_SCORE":E_score, "S_SCORE":S_score, "G_SCORE":G_score}
        final_df.loc[len(final_df)] = data_dc
    return final_df

df = base_scoring(r"C:\Users\Dell\Desktop\temp_esg")
# esg_score file
df.to_excel(r"C:\Users\Dell\Desktop\esg_score_copy.xlsx", index=False) 

In [8]:
df

,CIN,COMPANY_NAME,E_SCORE,S_SCORE,G_SCORE
0,L31300KA1987PLC013543,3M INDIA LIMITED,0.239,0.135,0.425
1,L65922RJ2011PLC034297,Aavas Financiers Limited,0.180,0.314,0.247
2,L32202KA1949PLC032923,ABB INDIA LIMITED,0.660,0.482,0.521
3,L51100GJ1993PLC019067,Adani Enterprises Limited (“AEL or the Company”),0.643,0.488,0.557
4,L40106GJ2015PLC082007,Adani Green Energy Limited,0.321,0.260,0.490
...,...,...,...,...,...
164,L29150TG1930PLC000576,VST INDUSTRIES LIMITED,0.483,0.378,0.404
165,L27100GJ1995PLC025609,WELSPUN CORP LIMITED,0.498,0.408,0.428
166,L17110GJ1985PLC033271,Welspun India Limited,0.511,0.383,0.381
167,L32102KA1945PLC020800,Wipro Limited,0.676,0.341,0.385


## industry classification
#### ONE TIME

In [10]:
# ONE TIME SCRIPT FOR putting IC's in base score file from factacay database
score_df = df

# It is the Facatacy Database containing CIN, Company_name, industrial_classification(factacy_classification)
dummy = pd.read_excel(r"C:\Users\Dell\Desktop\dummy_esg.xlsx")

# This is the ESG table containing CIN, Company_name, ESG_score.
# We need to add industrial classification of corresponding companies.
#score_df = pd.read_excel(r"C:\Users\Dell\Desktop\esg_score - Copy.xlsx")



# Dictionary mapping of Factacy_classification into ESG_Classification
mapping_dc = {
    'financials': ['fintech and financial services', 'insur tech'],
    
    'consumer discretionary industry': ['edtech', 'food tech',
                                        'real estate and construction services', 'retail',"ai tech",
                                        "spacetech","gaming, esports and digital entertainment",
                                        "online dating and matchmaking tech","travel and hospitality tech",
                                        "marketing services", "techwear"],
    
    'health care': ['healthtech', 'veterinary activities'],
    
    'industrials': ['autotech', 'home tech', 'farmtech', 'manufacturing'],
    
    'utilities': ['env tech', 'life science', 'transportation and logistics tech', 'personal care products and fashion tech'],
    
    'telecom': ['business services', 'media and entertainment', 'telecommunication'],
    
    'information technology': ['EMpTY'],
    
    'consumer staples': ["ecommerce tech","ride hailing services","apparel industry","professional services",
                         "travel and hospitality","consumer electronics","cloth manufacturing"],
}
def map_classification(df):
    list(map(lambda x: df["ESG_classification"].replace(mapping_dc[x], x, inplace=True), mapping_dc))




# Creating ESG_classification column if it doesn't exist. 
if not score_df.columns.str.contains("ESG_classification").any():
    score_df["ESG_classification"] = np.nan

# Finding the CIN of ESG table and searching them in factacy database to retrieve corresponding factacy_classification
industry = np.nan
for cin in score_df.CIN:
    #cin = cin.lower()
    if cin in set(dummy.CIN):
        industry = dummy[dummy['CIN'] == cin]["ESG_classification"]

        industry = industry.reset_index(drop=True)
        industry = industry[0]
        if isinstance(industry, str):
            industry=industry.lower().strip()
        
        idx = score_df[score_df["CIN"] == cin].index[0]
        score_df['ESG_classification'][idx] = industry
        
        # mapping Logic to map Factacy_classification to ESG_classification
        list(map(lambda x: score_df['ESG_classification'].replace(mapping_dc[x], x, inplace=True), mapping_dc))
        
# if there is no compound score column then add one
if not score_df.columns.str.contains("compound_score").any():
    score_df["compound_score"] = np.nan

In [13]:
score_df[5:]

,CIN,COMPANY_NAME,E_SCORE,S_SCORE,G_SCORE,ESG_classification
5,L63090GJ1998PLC034182,Adani Ports and Special Economic Zone Ltd.,0.663,0.407,0.625,utilities
6,L40300GJ2013PLC077803,Adani Transmission Limited (“ATL / the Company”),0.571,0.318,0.442,NaN
7,L24200MH1989PLC051018,Advanced Enzyme Technologies Limited,0.447,0.493,0.335,NaN
8,L65990MH1994PLC080451,Affle (India) Limited,0.053,0.190,0.403,NaN
9,L24230MH1979PLC022059,Ajanta Pharma Limited,0.434,0.428,0.589,utilities
...,...,...,...,...,...,...
164,L29150TG1930PLC000576,VST INDUSTRIES LIMITED,0.483,0.378,0.404,NaN
165,L27100GJ1995PLC025609,WELSPUN CORP LIMITED,0.498,0.408,0.428,NaN
166,L17110GJ1985PLC033271,Welspun India Limited,0.511,0.383,0.381,NaN
167,L32102KA1945PLC020800,Wipro Limited,0.676,0.341,0.385,NaN


## COMPOUND SCORE

In [22]:
# Here we expect that we get "|CIN|COMPANY_NAME|E_SCORE|S_SCORE|G_SCORE|ESG_classification|compound_score|" 7 columns
# Here we will:
#     -map factacy classification
#     -calculate compound score
#     -add new company information (CIN, COMPANY_NAME, ESG_classofocation) to base_score file


#Dictionary mapping of Factacy_classification into ESG_Classification
# mapping_dc = {
#     'financials': ['fintech and financial services', 'insur tech'],'consumer discretionary industry': ['edtech', 'food tech','real estate and construction services', 'retail',"ai tech","spacetech","gaming, esports and digital entertainment","online dating and matchmaking tech","travel and hospitality tech","marketing services", "techwear"], 'health care': ['healthtech', 'veterinary activities'],'industrials': ['autotech', 'home tech', 'farmtech', 'manufacturing'],'utilities': ['env tech', 'life science', 'transportation and logistics tech', 'personal care products and fashion tech'],'telecom': ['business services', 'media and entertainment', 'telecommunication'],'information technology': ['EMpTY'],'consumer staples': ["ecommerce tech","ride hailing services","apparel industry","professional services","travel and hospitality","consumer electronics","cloth manufacturing"],
# }

mapping_dc = {
    
    'financials': ["Fintech and Financial services", "Insurtech",], 

    'consumer discretionary industry': ["EdTech", "Food Tech", "Tech Wear", "real estate and construction services",
                                        "retail", "AI Tech ", "Spacetech", "Gaming, esports and Digital Entertainment",
                                        "Online dating and matchmaking Tech", "Travel and hospitality tech", "Marketing services"],

    'health care': ["HealthTech", "Veterinary activities"],

    'industrials': ["AutoTech","HomeTech","FarmTech","Manufacturing"],

    'utilities': ["Life Science", "environment tech", "Transportation and logistics tech", 
                  "personal care products and fashion tech"],

    'telecom': ["business services", "Media and Entertainment", "Telecommunication"],

    'information technology': ['EMpTY'],

    'consumer staples': ["Ecommerce Tech", "ride hailing services", "Apparel Industry", "Professional services",
                         "Travel and Hospitality", "Consumer Electronics", "Cloth Manufacturing"],
}


compound_score_dc = {"energy" : (0.4, 0.4, 0.2),
                     "materials" : (0.1, 0.7, 0.2),
                     "industrials" : (0.2, 0.4, 0.4),
                     "consumer discretionary industry" : (0.1, 0.4, 0.5),
                     "consumer staples" : (0.4, 0.2, 0.4),
                     "health care" : (0.2, 0.5, 0.3),
                     "financials" : (0.1, 0.5, 0.4),
                     "information technology" : (0.2, 0.4, 0.4),
                     "telecom" : (0.25, 0.35, 0.4),
                     "utilities" : (0.4, 0.18, 0.42),
                     np.nan:(0, 0, 0)
                    }

def compound_score(df):
    df["compound_score"] = df.apply(lambda row: row['E_SCORE']*compound_score_dc[row["ESG_classification"]][0]
                                          + row['S_SCORE']*compound_score_dc[row["ESG_classification"]][1]
                                          + row['G_SCORE']*compound_score_dc[row["ESG_classification"]][2], axis=1)

    
def map_classification(df):
    list(map(lambda x: df["ESG_classification"].replace(mapping_dc[x], x, inplace=True), mapping_dc))
    
    
# New file coming with CIN, NAME, Industry_classification
new_df = pd.read_excel(r"C:\Users\Dell\Desktop\cin_name_ic.xlsx")

#compund score base file
score_df = pd.read_csv(r"C:\Users\Dell\Desktop\compound_score.csv")

# Stripping columns
str_col = ["CIN", "COMPANY_NAME", "ESG_classification"]
for col in str_col:
    new_df[f"{col}"] = new_df[f"{col}"].str.strip()
    score_df[f"{col}"] = score_df[f"{col}"].str.strip()

# Adding those columns which are not present in new df(annu & ranny) that has CIN, NAME, IC
for col_name in score_df.columns:
    if col_name not in new_df.columns:
        new_df[col_name] = 0  #np.nan

new_df = new_df[score_df.columns]

for cin in new_df.CIN:
    if cin in set(score_df.CIN):
        new_df = new_df[new_df['CIN'] != f'{cin}']

map_classification(new_df)

score_df = pd.concat((score_df, new_df))
compound_score(score_df)

score_df.to_csv(r"C:\Users\Dell\Desktop\compound_score.csv", index=False)